In [0]:
# -*- encoding: utf-8 -*-
import random
import numpy as np
import time
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Lambda
import re



In [0]:
data_dir = './Batik300'
value_folder = os.listdir(data_dir)
total_value_folder = len(value_folder)


In [0]:
def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    
    return [ atof(c) for c in re.split(r'[+-_]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

In [0]:
value_folder.sort(key=natural_keys)


In [0]:
X_train, Y_train = [], []
X_test, Y_test = [], []


In [0]:
# Iterate through the list of subfolders (subjects)
# Idx is the subject ID
idx = 0
for a in range (0,300,6):
    for train in range(4):
        img = load_img(os.path.join(data_dir,value_folder[a+train]))
        img = img_to_array(img).astype('float32')/255
        X_train.append(img)
        Y_train.append(idx)
    for test in range(4,6):
        img = load_img(os.path.join(data_dir,value_folder[a+test]))
        img = img_to_array(img).astype('float32')/255
        X_test.append(img)
        Y_test.append(idx)
    idx=idx+1    

In [0]:

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)



In [46]:
print (len(X_train),len(X_test))

200 100


In [0]:
num_classes = len(np.unique(Y_train))


In [0]:
def euclidean_distance(vectors):
    vector1, vector2 = vectors
    sum_square = tf.reduce_sum(tf.square(vector1 - vector2), axis=1, keepdims=True)
    return tf.sqrt(tf.maximum(sum_square, tf.keras.backend.epsilon())) 

def contrastive_loss(Y_true,D):
    margin = 1
    loss= tf.reduce_mean(Y_true*tf.math.square(D)+(1 - Y_true)*tf.math.maximum((margin-D),0))
    return loss


In [0]:
def SiameseNetwork(ukuran):
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=ukuran),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='sigmoid')])
    
    return model
        

In [50]:
ukuran=(128,128,3)
tf.random.set_seed(100)
shared_network = SiameseNetwork(ukuran)
shared_network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 126, 126, 64)      1792      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 61, 61, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)            

In [51]:
input_top = tf.keras.Input(shape=ukuran)
input_bottom = tf.keras.Input(shape=ukuran)
output_top = shared_network(input_top)
output_bottom = shared_network(input_bottom)

distance = Lambda(euclidean_distance, output_shape=(1,))([output_top,output_bottom])  
model = tf.keras.Model(inputs=[input_top, input_bottom], outputs=distance)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 128)          1681408     input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1)            0           sequential_1[1][0]         

In [0]:
def create_pairs(X,Y, num_classes):
    pairs, labels = [], []
    # index of images in X and Y for each class
    class_idx = [np.where(Y==i)[0] for i in range(num_classes)]
    # The minimum number of images across all classes
    min_images = min(len(class_idx[i]) for i in range(num_classes)) - 1
    for c in range(num_classes):
        for n in range(min_images):
            # create positive pair
            img1 = X[class_idx[c][n]]
            img2 = X[class_idx[c][n+1]]
            pairs.append((img1, img2))
            labels.append(1)
            # create negative pair
            # list of classes that are different from the current class
            neg_list = list(range(num_classes))
            neg_list.remove(c)
            # select a random class from the negative list.
            # This class will be used to form the negative pair.
            neg_c = random.sample(neg_list,1)[0]
            img1 = X[class_idx[c][n]]
            img2 = X[class_idx[neg_c][n]]
            pairs.append((img1,img2))
            labels.append(0)
    return np.array(pairs), np.array(labels)


num_classes = len(np.unique(Y_train))
training_pairs, training_labels = create_pairs(X_train, Y_train,len(np.unique(Y_train)))
test_pairs, test_labels = create_pairs(X_test, Y_test,len(np.unique(Y_test)))

In [0]:
model.compile(loss=contrastive_loss, optimizer='adam')

In [54]:
model.fit([training_pairs[:, 0], training_pairs[:, 1]], training_labels,batch_size=64, epochs=20)

Epoch 1/20
5/5 [==============================] - 17s 3s/step - loss: 0.2740
Epoch 2/20
5/5 [==============================] - 18s 4s/step - loss: 0.1356
Epoch 3/20
5/5 [==============================] - 17s 3s/step - loss: 0.0934
Epoch 4/20
5/5 [==============================] - 17s 3s/step - loss: 0.0548
Epoch 5/20
5/5 [==============================] - 17s 3s/step - loss: 0.0533
Epoch 6/20
5/5 [==============================] - 21s 4s/step - loss: 0.0465
Epoch 7/20
5/5 [==============================] - 17s 3s/step - loss: 0.0338
Epoch 8/20
5/5 [==============================] - 17s 3s/step - loss: 0.0311
Epoch 9/20
5/5 [==============================] - 17s 3s/step - loss: 0.0231
Epoch 10/20
5/5 [==============================] - 18s 4s/step - loss: 0.0228
Epoch 11/20
5/5 [==============================] - 18s 4s/step - loss: 0.0164
Epoch 12/20
5/5 [==============================] - 18s 4s/step - loss: 0.0149
Epoch 13/20
5/5 [==============================] - 18s 4s/step - loss: 0.

In [0]:
data_for_matching = []

for a in range(0,300,6):
    img = load_img(os.path.join(data_dir,value_folder[a]))
    img = img_to_array(img).astype('float32')/255
    data_for_matching.append(img)
    
data_for_matching = np.array(data_for_matching)

print (len(data_for_matching))

In [0]:
kamus = dict({0: 'm-1', 1: 'm-2',
              2: 'm-3', 3: 'm-4',
              4: 'm-5', 5: 'm-6',
              6: 'm-7', 7: 'm-8',
              8: 'm-9', 9: 'm-10',
              10: 'm-11',11: 'm-12',
              12: 'm-13',13: 'm-14',
              14: 'm-15',15: 'm-16',
              16: 'm-17',17: 'm-18',
              18: 'm-19',19: 'm-20',
              
              20: 'm-21', 21: 'm-22',
              22: 'm-23', 23: 'm-24',
              24: 'm-25', 25: 'm-26',
              26: 'm-27', 27: 'm-28',
              28: 'm-29', 29: 'm-30',
              30: 'm-31', 31: 'm-32',
              32: 'm-33', 33: 'm-34',
              34: 'm-35', 35: 'm-36',
              36: 'm-37', 37: 'm-38',
              38: 'm-39', 39: 'm-40',
               
              40: 'm-41', 41: 'm-42',
              42: 'm-43', 43: 'm-44',
              44: 'm-45', 45: 'm-46',
              46: 'm-47', 47: 'm-48',
              48: 'm-49', 49: 'm-50'})


In [57]:
hitung_true=0
hitung_false=0
nama_kelas=1
for a in range(0,300,6):
    for b in range (4,6):
        hasil=[]
        img_test = load_img(os.path.join(data_dir,value_folder[a+b]))
        img_test = img_to_array(img_test).astype('float32')/255
        img_test = np.array(img_test)
        img_test = np.expand_dims(img_test, axis=0)
        for c in range(len(data_for_matching)):
            image_data = data_for_matching[c]
            image_data = np.expand_dims(image_data, axis=0)
            dissimilarity = model.predict([image_data, img_test])[0][0]
            hasil.append(dissimilarity)
            nilai_terendah = np.argmin(hasil)
        print ("File : ",value_folder[a+b],"model : ",kamus[nama_kelas-1],", predicted result : ",kamus[nilai_terendah])
        if nama_kelas-1 == nilai_terendah:
            hitung_true = hitung_true+1
        else:
            hitung_false=hitung_false+1
    nama_kelas=nama_kelas+1

File :  B1_5.jpg model :  m-1 , predicted result :  m-1
File :  B1_6.jpg model :  m-1 , predicted result :  m-1
File :  B2_5.jpg model :  m-2 , predicted result :  m-2
File :  B2_6.jpg model :  m-2 , predicted result :  m-2
File :  B3_5.jpg model :  m-3 , predicted result :  m-3
File :  B3_6.jpg model :  m-3 , predicted result :  m-3
File :  B4_5.jpg model :  m-4 , predicted result :  m-4
File :  B4_6.jpg model :  m-4 , predicted result :  m-4
File :  B5_5.jpg model :  m-5 , predicted result :  m-5
File :  B5_6.jpg model :  m-5 , predicted result :  m-5
File :  B6_5.jpg model :  m-6 , predicted result :  m-6
File :  B6_6.jpg model :  m-6 , predicted result :  m-6
File :  B7_5.jpg model :  m-7 , predicted result :  m-7
File :  B7_6.jpg model :  m-7 , predicted result :  m-7
File :  B8_5.jpg model :  m-8 , predicted result :  m-8
File :  B8_6.jpg model :  m-8 , predicted result :  m-8
File :  B9_5.jpg model :  m-9 , predicted result :  m-9
File :  B9_6.jpg model :  m-9 , predicted result

In [58]:
print ("True predicted : ",hitung_true," False predicted: ",hitung_false)


True predicted :  89  False predicted:  11


In [0]:
model.save('recognize_batik-siamese-final.h5')